### We're gonna do some preprocessing

I am doing this on Casper because I cannot figure out Asha, however I think the directory structure is the same once we navigate to the directory so maybe just start with

In [1]:
machine = 'casper' # 'casper' or 'asha'

I am going to write code that can load in data for a specific model and variable from the training. Maybe a function that can be called to do that.

In [2]:
# I/O / data wrangling
import glob
import numpy as np
import xarray as xr
import gc


In [3]:
if machine == 'casper':
    root_dir = "/glade/campaign/cgd/cas/asphilli/ForceSMIP/"
elif machine == 'asha':
    root_dir = "/barnes-scratch/DATA/ForceSMIP/"

In [4]:
cmipTable = {
    "pr": "Amon",
    "psl": "Amon",
    "tas": "Amon",
    "zmta": "Amon",
    "tos": "Omon",
    "siconc": "OImon",
    "monmaxpr": "Aday",
    "monmaxtasmax": "Aday",
    "monmintasmin": "Aday",
}
cmipVar = {
    "pr": "pr",
    "psl": "psl",
    "tas": "tas",
    "zmta": "ta",
    "tos": "tos",
    "siconc": "siconc",
    "monmaxpr": "pr",
    "monmaxtasmax": "tasmax",
    "monmintasmin": "tasmin",
}
evalPeriods = {
    "Tier1": ("1950-01-01", "2022-12-31"),
    "Tier2": ("1900-01-01", "2022-12-31"),
    "Tier3": ("1979-01-01", "2022-12-31"),
}
nlat = 72
nlon = 144

In [9]:
def load_model(model,var,timecut="Tier1",ntrainmems=10):

    filelist = root_dir + "Training/" + cmipTable[var] + "/" + var + "/" + model + "/" + var + "*.nc"
    filelist = glob.glob(filelist)
    filelist = sorted(filelist)

    timebds = evalPeriods[timecut]

    all_ens = []
    for ifile,file in enumerate(filelist[:ntrainmems]):
        ds = xr.open_dataset(file)
        varin = ds[cmipVar[var]]
        varcut = varin.sel(time=slice(timebds[0],timebds[1]))

        varcut=varcut.assign_coords({"variant":ifile+1})
        all_ens.append(varcut)

    all_ens = xr.concat(all_ens,dim="variant")
    
    return all_ens

In [10]:
var = "monmaxpr"
model = "CESM2"
timecut = "Tier1"
ntrainmems = 10

load_model(model,var)

<xarray.DataArray 'pr' (variant: 10, time: 876, lat: 72, lon: 144)>
array([[[[ 0.73147862,  0.72922166,  0.724346  , ...,  0.74718264,
           0.743738  ,  0.73618393],
         [ 1.07879291,  1.11988568,  1.04928392, ...,  1.04801551,
           1.04851523,  1.02761225],
         [ 2.04805864,  2.61381056,  2.59172303, ...,  1.13938127,
           0.99548724,  0.9336917 ],
         ...,
         [ 1.77672287,  1.92906775,  2.55471829, ...,  2.24104675,
           2.19507203,  1.97933921],
         [ 3.21271156,  3.31549096,  3.40167303, ...,  2.34361635,
           2.54100188,  3.03828837],
         [ 5.26799054,  5.16578391,  4.96587583, ...,  5.90715506,
           5.74304001,  5.46777001]],

        [[ 0.28348879,  0.27994491,  0.27570143, ...,  0.31252409,
           0.30328254,  0.29384194],
         [ 0.31649422,  0.38676907,  0.36846096, ...,  0.26113972,
           0.25464108,  0.26395386],
         [ 0.67121978,  0.63815568,  0.59831341, ...,  0.5782511 ,
           0.5993475 ,  0.70693945],
...
         [ 4.76994649,  5.03381897,  5.14511881, ...,  7.38593887,
           5.35015733,  4.24542301],
         [ 5.0415226 ,  4.51913031,  4.09113787, ...,  6.51602379,
           6.25319587,  5.66230915],
         [ 4.95328179,  4.94752824,  4.94440006, ...,  5.05638166,
           5.01759704,  4.97132672]],

        [[ 1.42187415,  1.41625704,  1.40333548, ...,  1.44885253,
           1.44287137,  1.43135152],
         [ 0.89175182,  0.91854542,  0.96217316, ...,  0.78971909,
           0.78740688,  0.83653617],
         [ 0.78243736,  0.7478881 ,  0.69360765, ...,  1.12999077,
           0.99094147,  0.85031351],
         ...,
         [ 7.93968043,  7.60463042,  7.19102034, ..., 11.20651941,
           9.49336604,  8.57377099],
         [ 4.41742862,  4.39733569,  4.34880097, ...,  4.32823717,
           4.37721728,  4.41878905],
         [ 1.49299458,  1.49397791,  1.49580644, ...,  1.49353774,
           1.48995696,  1.4891888 ]]]])
Coordinates:
  * time     (time) object 1950-01-16 12:00:00 ... 2022-12-16 12:00:00
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * variant  (variant) int64 1 2 3 4 5 6 7 8 9 10
Attributes:
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean
    units:         mm/day